In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import random
import webbrowser


# =============================
# Backend Logic (Placeholders)
# =============================


# Keys are illness names, values are detailed descriptions.
ILLNESS_DETAILS = {
    "Common Cold": "A viral infection of your nose and throat (upper respiratory tract). It's usually harmless, although it might not feel that way. Many types of viruses can cause a common cold. Symptoms include runny nose, sore throat, cough, congestion, slight body aches or a mild headache, sneezing, and low-grade fever.",
    "Influenza (Flu)": "Influenza is a contagious respiratory illness caused by influenza viruses that infect the nose, throat, and sometimes the lungs. It can cause mild to severe illness, and at times can lead to death. Symptoms are similar to a cold but often more severe, including high fever, body aches, extreme tiredness, and dry cough.",
    "Malaria": "Malaria is a mosquito-borne infectious disease that affects humans and other animals. It's caused by parasitic protozoans. Symptoms typically include fever, tiredness, vomiting, and headaches. In severe cases, it can cause yellow skin, seizures, coma, or death.",
    "Typhoid Fever": "Typhoid fever is a bacterial infection caused by Salmonella Typhi. It's often spread through contaminated food or water. Symptoms include high fever, weakness, abdominal pain, headache, and loss of appetite. Some people may have a rash.",
    "Allergies": "A condition in which the immune system reacts abnormally to a foreign substance that is typically not harmful to the body. Symptoms vary widely depending on the allergen but can include sneezing, itching, rashes, swelling, and difficulty breathing.",
    "Dengue Fever": "A mosquito-borne tropical disease caused by the dengue virus. Symptoms typically begin three to fourteen days after infection. These may include a high fever, headache, vomiting, muscle and joint pains, and a characteristic skin rash.",
    "Gastroenteritis": "Often called stomach flu, it's an inflammation of the stomach and intestines, typically caused by viral or bacterial infection. Symptoms include diarrhea, vomiting, abdominal cramps, and fever.",
    "Migraine": "A primary headache disorder characterized by recurrent headaches that are moderate to severe. Typically, the headaches affect one half of the head, are throbbing in nature, and last from 2 to 72 hours. Associated symptoms may include nausea, vomiting, and sensitivity to light, sound, or smell.",
    
}

# In a real application, this would be a more complex model (e.g., machine learning)
ILLNESS_SYMPTOM_MAP = {
    "Common Cold": ["fever", "cough", "sore throat", "runny nose", "sneezing", "fatigue", "headache"],
    "Influenza (Flu)": ["fever", "cough", "sore throat", "fatigue", "headache", "muscle pain", "chills", "vomiting", "diarrhea", "shortness of breath"],
    "Malaria": ["fever", "chills", "headache", "muscle pain", "fatigue", "nausea", "vomiting", "diarrhea"],
    "Typhoid Fever": ["fever", "abdominal pain", "headache", "fatigue", "loss of appetite", "diarrhea", "rash"],
    "Allergies": ["sneezing", "runny nose", "itching", "rash", "difficulty breathing", "sore throat"],
    "Dengue Fever": ["fever", "headache", "muscle pain", "joint pain", "rash", "nausea", "vomiting"],
    "Gastroenteritis": ["nausea", "vomiting", "diarrhea", "abdominal pain", "fever"],
    "Migraine": ["headache", "nausea", "vomiting", "sensitivity to light", "sensitivity to sound"],
}

# Recommended tests for different illnesses
ILLNESS_TESTS = {
    "Common Cold": ["No specific tests usually needed, diagnosis is clinical."],
    "Influenza (Flu)": ["Rapid Influenza Diagnostic Test (RIDT)", "Molecular Assays (RT-PCR)"],
    "Malaria": ["Malaria Blood Smear Test", "Rapid Diagnostic Test (RDT)"],
    "Typhoid Fever": ["Blood Culture", "Stool Culture", "Widal Test"],
    "Allergies": ["Skin Prick Test", "Blood Test (IgE antibody test)"],
    "Dengue Fever": ["Dengue NS1 Antigen Test", "IgM/IgG Antibody Test"],
    "Gastroenteritis": ["Stool Culture (for bacterial causes)"],
    "Migraine": ["Diagnosis is clinical, but imaging (MRI/CT) may be done to rule out other conditions."],
}

# Emergency symptoms that require immediate medical attention
EMERGENCY_SYMPTOMS = [
    "shortness of breath", "chest pain", "difficulty breathing", "seizures",
    "confusion", "severe abdominal pain", "loss of consciousness", "bloody stool",
    "sudden weakness or numbness on one side of the body"
]

class User:
    def __init__(self, name, age, gender, location, symptoms):
        self.name = name
        self.age = age
        self.gender = gender
        self.location = location
        self.symptoms = symptoms
        self.prediction = "Unknown Illness" # Default
        self.prediction_probabilities = {}

    def get_greeting(self):
        return f"Hello, {self.name}! Based on your information, here's a health prediction:"

    def predict(self):
        # This is a simplified prediction logic.
        # In a real-world scenario, you would use a machine learning model (e.g., Naive Bayes, SVM, Decision Tree)
        # trained on a large dataset of symptoms and diagnoses.

        # Calculate a "score" for each illness based on matching symptoms
        illness_scores = {illness: 0 for illness in ILLNESS_SYMPTOM_MAP}

        for symptom in self.symptoms:
            for illness, associated_symptoms in ILLNESS_SYMPTOM_MAP.items():
                if symptom in associated_symptoms:
                    illness_scores[illness] += 1

        # Determine the total score for normalization
        total_score = sum(illness_scores.values())

        if total_score == 0:
            self.prediction = "Unable to predict a specific illness."
            self.prediction_probabilities = {"No clear match": 1.0}
            return "Unable to predict based on the provided symptoms. Please consult a doctor for a proper diagnosis."

        # Calculate probabilities (simplified: score / total_score)
        self.prediction_probabilities = {
            illness: score / total_score
            for illness, score in illness_scores.items() if score > 0
        }

        # Find the most likely illness
        if self.prediction_probabilities:
            self.prediction = max(self.prediction_probabilities, key=self.prediction_probabilities.get)
        else:
            self.prediction = "No specific illness predicted."
            return "No specific illness predicted based on the provided symptoms."

        return "Prediction successful."

    def get_tests(self):
        return ILLNESS_TESTS.get(self.prediction, ["Consult a healthcare professional for recommended tests."])

    def check_emergencies(self):
        detected_emergencies = [symptom for symptom in self.symptoms if symptom in EMERGENCY_SYMPTOMS]
        return detected_emergencies

# =============================
# GUI Setup
# =============================
root = tk.Tk()
root.title("WellStart Health Checker")
root.geometry("900x650")
root.configure(bg="#2c3e50")
root.resizable(True, True)

# Modern Style Configuration
style = ttk.Style()
style.theme_use('clam') # 'clam' or 'alt' often provide a good base for customization

# Define custom colors
PRIMARY_COLOR = "#8e44ad" # Amethyst
ACCENT_COLOR = "#3498db" # Peter River
TEXT_COLOR = "#ecf0f1" # Clouds
INPUT_BG = "#34495e" # Wet Asphalt
INPUT_FG = "#ecf0f1" # Clouds
BUTTON_NORMAL = "#9b59b6" # Amethyst
BUTTON_ACTIVE = "#8e44ad" # Darker Amethyst
BUTTON_TEXT = "#ffffff" # White

# Styles using the defined colors, ensuring background consistency
style.configure('.', font=('Inter', 11), background=root['bg'], foreground=TEXT_COLOR)

style.configure("TFrame", background=root['bg'])
style.configure("TLabel", background=root['bg'], foreground=TEXT_COLOR)
style.configure("TButton",
                font=('Inter', 11, 'bold'),
                background=BUTTON_NORMAL,
                foreground=BUTTON_TEXT,
                borderwidth=0,
                focusthickness=3,
                focuscolor=ACCENT_COLOR,
                relief="flat")
style.map("TButton",
          background=[('active', BUTTON_ACTIVE), ('!disabled', BUTTON_NORMAL)],
          foreground=[('active', BUTTON_TEXT), ('!disabled', BUTTON_TEXT)])

style.configure("TEntry",
                fieldbackground=INPUT_BG,
                foreground=INPUT_FG,
                borderwidth=1,
                relief="solid",
                padding=5)

style.configure("TCombobox",
                fieldbackground=INPUT_BG,
                foreground=INPUT_FG,
                selectbackground=ACCENT_COLOR,
                selectforeground=INPUT_FG,
                background=INPUT_BG,
                arrowcolor=TEXT_COLOR,
                borderwidth=1,
                relief="solid",
                padding=5)
style.map('TCombobox',
          fieldbackground=[('readonly', INPUT_BG)],
          selectbackground=[('readonly', ACCENT_COLOR)],
          selectforeground=[('readonly', INPUT_FG)],
          background=[('readonly', INPUT_BG)],
          foreground=[('readonly', INPUT_FG)])


# All possible symptoms for dropdowns (ensure this list is comprehensive for your model)
all_symptoms = sorted(list(set(symptom for sublist in ILLNESS_SYMPTOM_MAP.values() for symptom in sublist)))


# Main Frame Layout
main_frame = ttk.Frame(root, padding="20 20 20 20", relief="flat", style="TFrame")
main_frame.pack(fill="both", expand=True, padx=20, pady=20)
main_frame.columnconfigure(0, weight=1)

main_frame.rowconfigure(4, weight=1) # Input area
main_frame.rowconfigure(6, weight=1) # Output text area

# Title & Welcome
title_label = ttk.Label(main_frame, text="WellStart Health Checker", font=("Inter", 24, "bold"), foreground=TEXT_COLOR, anchor="center")
title_label.grid(row=0, column=0, columnspan=2, pady=(0, 5), sticky="nsew")

welcome_label = ttk.Label(main_frame, text="Your Personal Health Checker", font=("Inter", 14), foreground=TEXT_COLOR, anchor="center")
welcome_label.grid(row=1, column=0, columnspan=2, pady=(0, 5), sticky="nsew")

# Disclaimer Label - Centered
disclaimer = ttk.Label(main_frame, text="Disclaimer! Please take note that it is always advisable to visit a Physician for accurate diagnosis and treatment.",
                       font=("Inter",12, "bold"), foreground="#e74c3c", justify="right")
disclaimer.grid(row=2, column=0, columnspan=2, pady=(5, 20), sticky="nsew")


# Inner Frame for User Info and Symptoms uses grid for better control
input_frame = ttk.Frame(main_frame, style="TFrame")
input_frame.grid(row=3, column=0, columnspan=2, sticky="nsew", pady=(10, 10))
input_frame.columnconfigure(0, weight=1)
input_frame.columnconfigure(1, weight=1)
input_frame.rowconfigure(0, weight=1)

# User Info Section
user_info_frame = ttk.Frame(input_frame, style="TFrame")
user_info_frame.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")
user_info_frame.columnconfigure(1, weight=1)

ttk.Label(user_info_frame, text="Your Details", font=("Inter", 14, "bold"), foreground=TEXT_COLOR).grid(row=0, column=0, columnspan=2, pady=(0, 10), sticky="w")
entries = {}

labels_info = ["Name", "Age", "Gender", "Location"]
for i, label_text in enumerate(labels_info):
    ttk.Label(user_info_frame, text=label_text + ":", font=("Inter", 11), foreground=TEXT_COLOR).grid(row=i+1, column=0, sticky="w", pady=5, padx=5)
    if label_text == "Gender":
        gender_var = tk.StringVar(value="Select Gender") # Set initial value
        gender_dropdown = ttk.Combobox(user_info_frame, textvariable=gender_var, values=["Male", "Female", "Other"], state="readonly", style="TCombobox")
        gender_dropdown.grid(row=i+1, column=1, sticky="ew", pady=5, padx=5)
        entries["gender"] = gender_dropdown
    else:
        ent = ttk.Entry(user_info_frame, style="TEntry")
        ent.grid(row=i+1, column=1, sticky="ew", pady=5, padx=5)
        entries[label_text.lower()] = ent
        if label_text == "Location":
            ent.insert(0, "e.g., Accra")
        # Ensure Age only accepts numbers
        if label_text == "Age":
             validate_age = root.register(lambda x: x.isdigit() or x == "")
             ent.config(validate="key", validatecommand=(validate_age, '%P'))


# Symptoms Section
symptoms_frame = ttk.Frame(input_frame, style="TFrame")
symptoms_frame.grid(row=0, column=1, padx=10, pady=10, sticky="nsew")
symptoms_frame.columnconfigure(0, weight=1)

ttk.Label(symptoms_frame, text="Select Symptoms:", font=("Inter", 14, "bold"), foreground=TEXT_COLOR).grid(row=0, column=0, pady=(0, 10), sticky="nsew")

symptom_vars = []
symptom_dropdowns = []

for i in range(4):
    var = tk.StringVar()
    symptom_vars.append(var)
    dropdown = ttk.Combobox(symptoms_frame, textvariable=var, values=all_symptoms, state="readonly", style="TCombobox")
    dropdown.grid(row=i+1, column=0, sticky="ew", pady=5, padx=5)
    dropdown.set(f"Symptom {i+1} (Optional)")
    symptom_dropdowns.append(dropdown)


# Centered Button Frame (Submit/Reset)
button_frame = ttk.Frame(main_frame, style="TFrame")
button_frame.grid(row=4, column=0, columnspan=2, pady=(5, 5), sticky="nsew")
button_frame.columnconfigure(0, weight=1)
button_frame.columnconfigure(1, weight=1)

submit_btn = ttk.Button(button_frame, text="✅ Submit", command=lambda: submit(), style="TButton")
submit_btn.grid(row=0, column=0, padx=10, pady=10, sticky="e")

reset_btn = ttk.Button(button_frame, text="↺ Reset", command=lambda: reset(), style="TButton")
reset_btn.grid(row=0, column=1, padx=10, pady=10, sticky="w")

# Output Display - Using ttk.Text for better styling control
output_text = tk.Text(main_frame, height=6, wrap="word", bg=INPUT_BG, fg=INPUT_FG,
                     font=("Inter", 11), relief="solid", bd=1, padx=10, pady=10)
output_text.grid(row=5, column=0, columnspan=2, sticky="nsew", padx=10, pady=10)
output_text.config(state="disabled")

# --- Loading Bar Definition ---
style.configure("TProgressbar", thickness=15, troughcolor=INPUT_BG, background=ACCENT_COLOR)
# loading_bar is intentionally placed as a child of main_frame
loading_bar = ttk.Progressbar(main_frame, orient="horizontal", mode="indeterminate", style="TProgressbar")
# ------------------------------


# New Buttons Frame (Hospital, Pharmacy, Lab)
action_buttons_frame = ttk.Frame(main_frame, style="TFrame")
action_buttons_frame.grid(row=6, column=0, columnspan=2, pady=(5, 5), sticky="nsew")
action_buttons_frame.columnconfigure(0, weight=1)
action_buttons_frame.columnconfigure(1, weight=1)
action_buttons_frame.columnconfigure(2, weight=1)

# Placeholder functions for the new buttons
def open_hospital_info():
    location_val = entries['location'].get().strip()
    if location_val and location_val != "e.g., Accra":
        messagebox.showinfo("Hospital Info", f"Searching for nearby hospitals in {location_val}...")
        # Placeholder URL for map search
        webbrowser.open_new_tab(f"https://www.google.com/maps/search/hospitals+in+{location_val.replace(' ', '+')}")
    else:
        messagebox.showwarning("Location Needed", "Please enter your location to find nearby hospitals.")

def open_pharmacy_info():
    location_val = entries['location'].get().strip()
    if location_val and location_val != "e.g., Accra":
        messagebox.showinfo("Pharmacy Info", f"Searching for nearby pharmacies in {location_val}...")
        # Placeholder URL for map search
        webbrowser.open_new_tab(f"https://www.google.com/maps/search/pharmacies+in+{location_val.replace(' ', '+')}")
    else:
        messagebox.showwarning("Location Needed", "Please enter your location to find nearby pharmacies.")

def open_lab_info():
    location_val = entries['location'].get().strip()
    if location_val and location_val != "e.g., Accra":
        messagebox.showinfo("Lab Info", f"Searching for nearby labs in {location_val}...")
        # Placeholder URL for map search
        webbrowser.open_new_tab(f"https://www.google.com/maps/search/medical+laboratories+in+{location_val.replace(' ', '+')}")
    else:
        messagebox.showwarning("Location Needed", "Please enter your location to find nearby labs.")

ttk.Button(action_buttons_frame, text="🏥 Hospital", command=open_hospital_info, style="TButton").grid(row=0, column=0, padx=5, pady=5, sticky="ew")
ttk.Button(action_buttons_frame, text="💊 Pharmacy", command=open_pharmacy_info, style="TButton").grid(row=0, column=1, padx=5, pady=5, sticky="ew")
ttk.Button(action_buttons_frame, text="🔬 Lab", command=open_lab_info, style="TButton").grid(row=0, column=2, padx=5, pady=5, sticky="ew")


# Emergency Number Display (Clickable)
emergency_number_label = ttk.Label(main_frame, text="🚨 Emergency Number: 112", font=("Inter", 11, "bold", "underline"),
                                     foreground="#e74c3c", cursor="hand2")
emergency_number_label.grid(row=7, column=0, columnspan=2, pady=(5, 0), sticky="nsew")
emergency_number_label.bind("<Button-1>", lambda e: messagebox.showinfo("Emergency Call", "Please dial 112 for immediate medical attention in Ghana."))


# Actions
def reset():
    """Resets all input fields and the output display to their initial state."""
    for ent in entries.values():
        if isinstance(ent, ttk.Combobox):
            # Do not use ent.set("") as it clears the initial value which is 'Select Gender'
            if ent == entries["gender"]:
                ent.set("Select Gender")
            else:
                ent.set("") 
        else:
            ent.delete(0, tk.END)
            if ent == entries["location"]:
                ent.insert(0, "e.g., Accra")

    for i, var in enumerate(symptom_vars):
        symptom_dropdowns[i]['values'] = all_symptoms # Ensure dropdowns use the updated symptom list
        var.set(f"Symptom {i+1} (Optional)")


    output_text.config(state="normal")
    output_text.delete(1.0, tk.END)
    output_text.config(state="disabled")

def submit():
    """Gathers user input, performs validation, and initiates the prediction process."""
    name = entries["name"].get().strip()
    age = entries["age"].get().strip()
    gender = entries["gender"].get().strip()
    loc = entries["location"].get().strip()

    symptoms = [var.get().lower().strip() for var in symptom_vars if var.get() and "optional" not in var.get().lower() and var.get() in all_symptoms]

    # Input Validation
    if not name or not age or not gender or gender == "Select Gender" or not loc or loc == "e.g., Accra":
        messagebox.showerror("Missing Info", "Please fill in all user details accurately.")
        return

    try:
        age = int(age)
        if not (0 < age < 120): # Basic age validation
            messagebox.showerror("Input Error", "Please enter a realistic age (e.g., between 1 and 120).")
            return
    except ValueError:
        messagebox.showerror("Input Error", "Please enter a valid number for Age.")
        return

    # Enforce two-symptom minimum for a more accurate prediction
    if len(symptoms) < 2:
        messagebox.showwarning("Missing Symptoms", "Please select at least two symptoms to get a more accurate prediction.")
        return

    # Start loading animation and disable buttons
    loading_bar.grid(row=8, column=0, columnspan=2, sticky="ew", padx=10, pady=10)
    loading_bar.start(10)
    submit_btn.config(state="disabled")
    reset_btn.config(state="disabled")
    output_text.config(state="normal")
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, "Processing your symptoms...\n")
    output_text.config(state="disabled")
    root.update_idletasks() # Update the GUI to show loading message immediately

    # Schedule the prediction to run after a short delay to allow loading animation to show
    # The fix for the race condition is primarily within perform_prediction
    root.after(1500, lambda: perform_prediction(name, age, gender, loc, symptoms))


def perform_prediction(name, age, gender, location, symptoms):
    """
    Executes the prediction logic and updates the GUI with results.
    Includes error handling for TclError to prevent crashes if the window is closed.
    """
    
    # === CRITICAL FIX: Graceful exit if the main application window is closed ===
    if not root.winfo_exists():
        return
    # ===========================================================================

    user = User(name, age, gender, location, symptoms)
    pred_status = user.predict() # This will update user.prediction and user.prediction_probabilities

    # Wrap all widget operations in a try-except block to handle closing during execution
    try:
        output_text.config(state="normal")
        output_text.delete(1.0, tk.END)

        output_text.insert(tk.END, user.get_greeting() + "\n")

        if "Unable to predict" in pred_status or "No specific illness predicted" in pred_status:
            output_text.insert(tk.END, f"\n{pred_status}\n")
        else:
            output_text.insert(tk.END, f"\n🤖 **Most Likely Illness:** {user.prediction}\n")

            # Display top 3 predicted illnesses with percentages
            output_text.insert(tk.END, "\n📊 **Prediction Probabilities (Top 3):**\n")

            sorted_predictions = sorted(user.prediction_probabilities.items(), key=lambda item: item[1], reverse=True)

            for illness, prob in sorted_predictions[:3]:
                output_text.insert(tk.END, f"  - {illness}: {prob:.2%}\n") # Format as percentage

            output_text.insert(tk.END, f"\nℹ️ **About {user.prediction}:** {ILLNESS_DETAILS.get(user.prediction, 'No detailed information available for this illness.')}\n")

            tests = user.get_tests()
            if tests:
                output_text.insert(tk.END, f"\n🧪 **Recommended Tests for {user.prediction}:**\n")
                for test in tests:
                    output_text.insert(tk.END, f" - {test}\n")

            emergencies = user.check_emergencies()
            if emergencies:
                output_text.insert(tk.END, f"\n🚨 **Emergency Symptoms Detected:** {', '.join(emergencies)}\n")
                output_text.insert(tk.END, "📞 **Please seek immediate medical attention!**\n")
                output_text.insert(tk.END, "🚑 **Suggestion:** Call emergency services.\n")
                output_text.insert(tk.END, "☎️ **Dial emergency number:** 112\n")

        output_text.config(state="disabled")

        # Stop loading animation and re-enable buttons
        loading_bar.stop()
        loading_bar.grid_forget()
        submit_btn.config(state="normal")
        reset_btn.config(state="normal")
        
    except tk.TclError:
        # Catching the TclError prevents the program from crashing if the window
        # is destroyed mid-function execution.
        pass

# Initial reset to set placeholders correctly and populate dropdowns when the app starts
reset()

root.mainloop()